In [23]:
import time
from janus_client import Client, Session, Service

from FHGDemo import setup, run_workflow

JANUS_URL="https://nersc-srv-1.testbed100.es.net:5050"

user = 'kissel'
mykey = 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC/DAAkHubwAfjOSVG67tvwvUrESRoig+OAk5htH0mLK7QaN4gTv/C3ritJH3PkRzIJyOAJVS1ssjiBtMpI8Lb8NZCYKDZdpH/803cPpd2KqI2oAWRZQcjamKYs6SlM9r380wZ069r143ldaWPU3CJ6ye2+yx0Gifdivutnk5eQ1DxdK/VbukMzWeDdO8T1QXHxqP6lRy5PUkDKlY6Ec8i8YLYijhBoMlK0NL23d9ZDrVFKYe4D9OBp0CCKzkz9a7qgLOoYvCGRzN7BXIztJkYzziFAYn4Ru0RTyHM81V8jp4H0E2EM7BDtUUBac/sk7pJS7w2rIRZtPDsFA7qFvw+1 dtnaas@a83afe25e688'

client = Client(JANUS_URL, auth=(user, 'd3m0p4ss'))


sess_rdma1 = client.getSession()
srv1 = Service(instances=['nersc-tbn-6'], image='dtnaas/ofed:5.4-3', profile='lbnl-400g-1', username=user, public_key=mykey)
srv2 = Service(instances=['nersc-tbn-7'], image='dtnaas/ofed:5.4-3', profile='lbnl-400g-2', username=user, public_key=mykey)
sess_rdma1.addService(srv1)
sess_rdma1.addService(srv2)


sess_rdma2 = client.getSession()
srv1 = Service(instances=['nersc-srv-1'], image='dtnaas/ofed:5.4-3', profile='lbnl-400g-1', username=user, public_key=mykey)
srv2 = Service(instances=['nersc-dtnaas-1'], image='dtnaas/ofed:5.4-3', profile='lbnl-400g-2', username=user, public_key=mykey)
sess_rdma2.addService(srv1)
sess_rdma2.addService(srv2)

sess_escp = client.getSession()
srv = Service(instances=['nersc-tbn-1', 'nersc-tbn-2'],
              image='dtnaas/tools',
              profile='lbnl-400g-host',
              username=user,
              public_key=mykey)
sess_escp.addService(srv)

print (f"{sess_rdma1}\n\n{sess_rdma2}\n\n{sess_escp}")


id: 90d02619-5ac6-45b6-ba49-65680467e542
allocated: False
requests: [{'instances': ['nersc-tbn-6'], 'image': 'dtnaas/ofed:5.4-3', 'profile': 'lbnl-400g-1', 'kwargs': {'USER_NAME': 'kissel', 'PUBLIC_KEY': 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC/DAAkHubwAfjOSVG67tvwvUrESRoig+OAk5htH0mLK7QaN4gTv/C3ritJH3PkRzIJyOAJVS1ssjiBtMpI8Lb8NZCYKDZdpH/803cPpd2KqI2oAWRZQcjamKYs6SlM9r380wZ069r143ldaWPU3CJ6ye2+yx0Gifdivutnk5eQ1DxdK/VbukMzWeDdO8T1QXHxqP6lRy5PUkDKlY6Ec8i8YLYijhBoMlK0NL23d9ZDrVFKYe4D9OBp0CCKzkz9a7qgLOoYvCGRzN7BXIztJkYzziFAYn4Ru0RTyHM81V8jp4H0E2EM7BDtUUBac/sk7pJS7w2rIRZtPDsFA7qFvw+1 dtnaas@a83afe25e688'}}, {'instances': ['nersc-tbn-7'], 'image': 'dtnaas/ofed:5.4-3', 'profile': 'lbnl-400g-2', 'kwargs': {'USER_NAME': 'kissel', 'PUBLIC_KEY': 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC/DAAkHubwAfjOSVG67tvwvUrESRoig+OAk5htH0mLK7QaN4gTv/C3ritJH3PkRzIJyOAJVS1ssjiBtMpI8Lb8NZCYKDZdpH/803cPpd2KqI2oAWRZQcjamKYs6SlM9r380wZ069r143ldaWPU3CJ6ye2+yx0Gifdivutnk5eQ1DxdK/VbukMzWeDdO8T1QXHxqP6lRy5PUkDKlY6Ec8i8YL

In [24]:
res = sess_rdma1.start()
print (sess_rdma1.status())

res = sess_rdma2.start()
print (sess_rdma2.status())

res = sess_escp.start()
print (sess_escp.status())

id: 46, service: nersc-tbn-6, errors: []
id: 46, service: nersc-tbn-7, errors: []

id: 47, service: nersc-srv-1, errors: []
id: 47, service: nersc-dtnaas-1, errors: []

id: 48, service: nersc-tbn-1, errors: []
id: 48, service: nersc-tbn-2, errors: []



In [25]:
print (f"-- RDMA Src Endpoints:\n{sess_rdma1.endpoints()}")
print (f"-- RDMA Dst Endpoints:\n{sess_rdma2.endpoints()}")
print (f"-- ESCP Endpoints:\n{sess_escp.endpoints()}")

-- RDMA Src Endpoints:
nersc-tbn-6: nersc-tbn-6.testbed100.es.net:30000
nersc-tbn-7: nersc-tbn-7.testbed100.es.net:30001
-- RDMA Dst Endpoints:
nersc-srv-1: nersc-srv-1.testbed100.es.net:30000
nersc-dtnaas-1: nersc-dtnaas-1.testbed100.es.net:30001
-- ESCP Endpoints:
nersc-tbn-1: nersc-tbn-1.testbed100.es.net:30000
nersc-tbn-2: nersc-tbn-2.testbed100.es.net:30001


In [42]:
import sys
import time
import logging
import subprocess
import threading
from IPython.display import display
import ipywidgets as widgets
from subprocess import PIPE, STDOUT
from ipaddress import IPv4Network, IPv4Address


log = logging.getLogger("FHGDemo")
log.setLevel(logging.INFO)

def run_host_cmd(host, cmd, ofname=None, stop=None, interactive=False, out=None):
    log.debug(f"Running \"{cmd}\" on \"{host}\", with output to file \"{ofname}\"")
    parts = host.split(":")
    if len(parts) > 1:
        initcmd = ["ssh", "-t", "-o", "StrictHostKeyChecking=no", "-p", parts[1], parts[0]]
    else:
        initcmd = ["ssh", "-t", "-o", "StrictHostKeyChecking=no", host]
    rcmd = initcmd + cmd.split(" ")
    log.debug(rcmd)
    try:
        proc = subprocess.Popen(rcmd, stdout=PIPE)
        if interactive:
            for p in proc.stdout:
                if out:
                    out.append_stdout(p.decode('utf-8'))
                else:
                    print (p.decode('utf-8'), end='')
    except Exception as e:
        log.info(f"Error running {rcmd}: {e}")
        return
    outs = None
    errs = None
    if stop:
        while not stop():
            time.sleep(1)
        proc.terminate()
        try:
            proc.wait(timeout=2)
        except TimeoutExpired:
            proc.kill()
    else:
        outs, errs = proc.communicate()
        proc.terminate()

    if not ofname:
        return
    try:
        f = open(ofname, 'wb')
        if not outs:
            outs = proc.stdout.read()
        f.write(outs)
        f.close()
    except Exception as e:
        log.error(f"Could not write output for \"{ofname}\": {e}")
        return

def setup(sess):
    res = sess.status().json()[0]
    for idx,ep in sess.endpoints().json().items():
        for k,v in res.items():
            if idx in v['services']:
                svc = v['services'][idx][0]
                try:
                    v4net = IPv4Network(svc['data_ipv4']+'/24', strict=False)
                    cmd = None
                    if str(v4net[1]) == "10.33.1.1":
                        cmd = "sudo route add -net 10.33.2.0/24 gw 10.33.1.1"
                        sess.dst = ep
                        sess.dst_ip = svc['data_ipv4']
                        sess.dst_gw = "10.33.1.1"
                    elif str(v4net[1]) == "10.33.2.1":
                        cmd = "sudo route add -net 10.33.1.0/24 gw 10.33.2.1"
                        sess.src = ep
                        sess.src_ip = svc['data_ipv4']
                        sess.src_gw = "10.33.2.1"
                    if cmd:
                        run_host_cmd(ep, cmd)
                except:
                    continue

def run_workflow(sess):
    out = widgets.Output()
    display(out)
    #out.append_stdout(f"Running workflow on {sess.src} and {sess.dst}")
    if getattr(sess, "src"):
        dst_cmd = f"ping -c 2 {sess.src_gw}; xfer_test -s -r -d 128"
        dst_thr = threading.Thread(target=run_host_cmd, args=(sess.dst, dst_cmd))
        src_cmd = f"ping -c 2 {sess.dst_gw}; stdbuf -oL xfer_test -c {sess.dst_ip} -t 12000 -i 5 -r -a 1 -o 24 -d 112"
        src_thr = threading.Thread(target=run_host_cmd, args=(sess.src, src_cmd, None, None, True, out))
        dst_thr.start()
        time.sleep(1)
        src_thr.start()
        return [src_thr, dst_thr]

#setup(sess_rdma1)
#setup(sess_rdma2)
#setup(sess_escp)

display('Display in main thread')
ret = run_workflow(sess_rdma1)
#setup(sess_rdma1)
#setup(sess_rdma2)
#setup(sess_escp)

'Display in main thread'

Output()

In [43]:
run_workflow(sess_rdma1)

Output()

[<Thread(Thread-37, started 139715890116160)>,
 <Thread(Thread-36, started 139716451870272)>]

In [19]:
run_workflow(sess_rdma2) 

[<Thread(Thread-11, started 139715881723456)>,
 <Thread(Thread-10, started 139716443477568)>]

PING 10.33.1.1 (10.33.1.1) 56(84) bytes of data.
64 bytes from 10.33.1.1: icmp_seq=1 ttl=64 time=88.4 ms
[30.0-35.0 sec]	      58.25 GB	      93.20 Gb/s
64 bytes from 10.33.1.1: icmp_seq=2 ttl=64 time=44.2 ms

--- 10.33.1.1 ping statistics ---
2 packets transmitted, 2 received, 0% packet loss, time 2ms
rtt min/avg/max/mdev = 44.157/66.254/88.352/22.098 ms
Using a SLaBS buffer of size 16777216 with 1 partitions of size 16777216
112: | port=18515 | ib_port=1 | tx_depth=112 | sl=0 | duplex=0 | cma=1 |
Created SLAB buffer with SIZE: 16777216 PARTITIONS: 1
raddr: 0x7f0892bfe000, laddr: 0x7f98a6fe6000, size: 16777216
Metadata exchange complete
[35.0-40.0 sec]	      58.25 GB	      93.20 Gb/s
[0.0-5.0 sec]	      93.83 GB	     150.13 Gb/s
[40.0-45.0 sec]	      58.25 GB	      93.20 Gb/s
[5.0-10.0 sec]	     105.68 GB	     169.09 Gb/s
[45.0-50.0 sec]	      58.25 GB	      93.20 Gb/s
[10.0-15.0 sec]	     105.68 GB	     169.09 Gb/s
[50.0-55.0 sec]	      58.25 GB	      93.20 Gb/s
[15.0-20.0 sec]	     

In [ ]:
run_workflow(sess_escp)

In [ ]:
sess_rdma1.stop()
sess_rdma2.stop()
sess_escp.stop()

In [20]:
sess_rdma1.destroy()
sess_rdma2.destroy()
sess_escp.destroy()

[70.0-75.0 sec]	     251.66 MB	     402.65 Mb/s
[75.0-80.0 sec]	     234.88 MB	     375.81 Mb/s
[80.0-85.0 sec]	     234.88 MB	     375.81 Mb/s
